In [1]:
!pip install torchattacks
!pip install pandas
!pip install openpyxl
!pip install numpy
!pip install plot 
!pip install numpy


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
import torch.nn as nn
import torchvision
import torchattacks
from torch.optim import lr_scheduler
import time

In [2]:
import Sources.DefaultModel as Util
import Sources.GTSRBModel as GTSRBModel
import Sources.TestUtils as TestUtils

c:\Sources\Python311\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Sources\Python311\Lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://g

In [5]:
Device = Util.HardwareIdentification()
DimImage = 50
TrainLoader,ValidationLoader, TestLoader = GTSRBModel.ImportData(1, DimImage)
print(len(TrainLoader.dataset), len(ValidationLoader.dataset), len(TestLoader.dataset))

cuda
50
31367
7842
12630
31367 7842 12630


In [ ]:
EPOCHS = 5
LEARNING_RATE = 0.0008
INPUT_DIM = 3*DimImage*DimImage
OUTPUT_DIM = 43
Model = GTSRBModel.GTSRB_MODEL(INPUT_DIM,OUTPUT_DIM, Device).to(Device)

optimizer = torch.optim.Adam(params=Model.parameters(),lr=LEARNING_RATE)
lr_s = lr_scheduler.LinearLR(optimizer,start_factor=1.0,end_factor=0.5,total_iters=10)
loss = nn.CrossEntropyLoss()

In [ ]:
#%%script false --no-raise-error
Model.compile(train_data=TrainLoader,validation_data=ValidationLoader,epochs=EPOCHS,loss_function=loss,optimizer=optimizer,learning_rate_scheduler=lr_s)
ModelPath = 'Models\\ModelSignal.torch'
torch.save(Model.state_dict(), ModelPath)

In [ ]:
Model.load_state_dict(torch.load('Models\\ModelSignal.torch'))

In [ ]:
Util.TestLoop(TestLoader, Model, Device)
X, y = next(iter(TestLoader))

In [ ]:
Coef=5
Activation=4
Prob = 2
Similarity = 0.2
Init=0
lr = 0.2
Steps = 100

data = []
start_time = time.time()
data = TestUtils.TestLoopC(Model, Steps, lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "Results\\GTSRB\\ComputationTimeResults.txt")
TestUtils.SaveData(data, "Results\\GTSRB\\Results.xlsx")

In [ ]:
TestUtils.SaveImages("Results\\GTSRB\\InitialImages\\", X, y)

In [ ]:
Coef=5
Activation=4
Prob = 2
Similarity = 0.2
Init=0
lr = 0.2
Steps = 10000

TestUtils.SaveLoopC(Model, Steps, lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5, "Results\\GTSRB\\Images\\")

In [ ]:
Coef=5
Activation=4
Prob = 2
Similarity = 0.2
Init=0
lr = 0.2

TestedIterations = [10, 100]

for i in range((len(TestedIterations))):
    data = []
    start_time = time.time()
    data = TestUtils.TestLoopC(Model, TestedIterations[i], lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5)
    end_time = time.time()

    TestUtils.SaveTime(start_time, end_time, "Results\\GTSRB\\TimeXS" + str(TestedIterations[i]) +".txt")
    TestUtils.SaveData(data, "Results\\GTSRB\\BestXS" + str(TestedIterations[i]) + ".xlsx")
    

In [ ]:
dataJSMA = []

start_time = time.time()
Data = TestUtils.TestJSMA(1,1, Model, X, y, Device)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "Results\\GTSRB\\TimeJSMAS.txt")
TestUtils.SaveData(Data, "Results\\GTSRB\\BestJSMAS.xlsx")

In [ ]:
TestUtils.SaveJSMA(1,1, Model, X, y, Device, "Results\\GTSRB\\JSMAImages\\")